In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as func
import torchvision
from torchsummary import summary
import torch.utils as utils
from torchvision.ops.roi_align import RoIAlign
import numpy as np
import os
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
import json
import pandas as pd
import cv2
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from tqdm import tqdm
import shutil
from xml.dom import minidom
from scipy.ndimage.interpolation import map_coordinates
from scipy.ndimage.filters import gaussian_filter
from sklearn.feature_extraction.image import extract_patches_2d
from skimage.draw import polygon
from scipy.io import savemat
import torch.utils.data
from PIL import Image
from pathlib import Path
import sklearn

In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1jiZIr64TaavSvLKiq1a-wGksskB18zR0' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1jiZIr64TaavSvLKiq1a-wGksskB18zR0" -O monuseg.zip && rm -rf /tmp/cookies.txt


In [ ]:
!unzip "/content/monuseg.zip"

In [ ]:
%%shell
# Download TorchVision repo to use some files from
# references/detection
git clone https://github.com/pytorch/vision.git
cd vision
git checkout v0.3.0

cp references/detection/utils.py ../
cp references/detection/transforms.py ../
cp references/detection/coco_eval.py ../
cp references/detection/engine.py ../
cp references/detection/coco_utils.py ../

pip install cython
# Install pycocotools, the version by default in Colab
# has a bug fixed in https://github.com/cocodataset/cocoapi/pull/354
pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

In [ ]:
def generate_masks(shape, xml_file):
	"""
	Given the image shape and path to annotations (xml file),
	generate a bit mask with the region inside a contour being white
	shape: The image shape on which bit mask will be made
	xml_file: path relative to the current working directory
	where the xml file is present
	Returns: A image of given shape with region inside contour being white..
	"""
	# DOM object created by the minidom parser
	xDoc = minidom.parse(xml_file)

	# List of all Region tags
	regions = xDoc.getElementsByTagName('Region')

	# List which will store the vertices for each region
	xy = []
	for region in regions:
		vertices = region.getElementsByTagName('Vertex')

		# The vertices of a region will be stored in a array
		vw = np.zeros((len(vertices), 2))

		for index, vertex in enumerate(vertices):
			# Storing the values of x and y coordinate after conversion
			vw[index][0] = float(vertex.getAttribute('X'))
			vw[index][1] = float(vertex.getAttribute('Y'))

		# Append the vertices of a region
		xy.append(np.int32(vw))

	# Creating a completely black image
	mask = np.zeros(shape, np.int32)

	# For each contour, fills the area inside it
	# Warning: If a list of contours is passed, overlapping regions get buggy output
	# Comment out the below line to check, and if the bug is fixed use this
	# cv2.drawContours(mask, xy, -1, (255,255,255), cv2.FILLED)
	j=1
	for i,contour in enumerate(xy):
		if(xml_file.find('-')>=0):
			r,c = polygon(np.array(xy[i])[:,1],np.array(xy[i])[:,0],shape=shape)
		else:
			r,c = np.array(xy[i])[:,1],np.array(xy[i])[:,0]

		mask[r,c]=i+1	
	return mask

In [ ]:
def make_patches_img(img_inp,filename,folder):
    trans = transforms.ToPILImage()
    trans_tensor = transforms.ToTensor()
    img = trans_tensor(img_inp)
    kernel = 250
    ## num = 16
    patch1 = trans(img[:,0:kernel, 0:kernel])
    patch2 = trans(img[:,0:kernel, kernel:500])
    patch3 = trans(img[:,0:kernel, (kernel+250):750])
    patch4 = trans(img[:,0:kernel, (kernel+500):1000])
    patch5 = trans(img[:,kernel:500, 0:kernel])
    patch6 = trans(img[:,(kernel+250):750, 0:kernel])
    patch7 =  trans(img[:,(kernel+500):1000, 0:kernel])
    patch8 = trans(img[:,kernel:500, kernel:500])
    patch9 = trans(img[:,kernel:500, (kernel+250):750])
    patch10 = trans(img[:,kernel:500, (kernel+500):1000])
    patch11 = trans(img[:,(kernel+250):750, kernel:500])
    patch12 = trans(img[:,(kernel+500):1000, kernel:500])
    patch13 = trans(img[:,(kernel+250):750, (kernel+250):750])
    patch14 = trans(img[:,(kernel+250):750, (kernel+500):1000])
    patch15 = trans(img[:,(kernel+500):1000, (kernel+250):750])
    patch16 = trans(img[:,(kernel+500):1000,(kernel+500):1000])

    #save the images in seperate files in a new folder
    path1 = os.path.join(folder,'Tissue Images',"1_"+filename)
    patch1.save(path1)
    path2 = os.path.join(folder,'Tissue Images',"2_"+filename)
    patch2.save(path2)
    path3 = os.path.join(folder,'Tissue Images',"3_"+filename)
    patch3.save(path3)
    path4 = os.path.join(folder,'Tissue Images',"4_"+filename)
    patch4.save(path4)
    path5 = os.path.join(folder,'Tissue Images',"5_"+filename)
    patch5.save(path5)
    path6 = os.path.join(folder,'Tissue Images',"6_"+filename)
    patch6.save(path6)
    path7 = os.path.join(folder,'Tissue Images',"7_"+filename)
    patch7.save(path7)
    path8 = os.path.join(folder,'Tissue Images',"8_"+filename)
    patch8.save(path8)
    path9 = os.path.join(folder,'Tissue Images',"9_"+filename)
    patch9.save(path9)
    path10 = os.path.join(folder,'Tissue Images',"10_"+filename)
    patch10.save(path10)
    path11 = os.path.join(folder,'Tissue Images',"11_"+filename)
    patch11.save(path11)
    path12 = os.path.join(folder,'Tissue Images',"12_"+filename)
    patch12.save(path12)
    path13 = os.path.join(folder,'Tissue Images',"13_"+filename)
    patch13.save(path13)
    path14 = os.path.join(folder,'Tissue Images',"14_"+filename)
    patch14.save(path14)
    path15 = os.path.join(folder,'Tissue Images',"15_"+filename)
    patch15.save(path15)
    path16 = os.path.join(folder,'Tissue Images',"16_"+filename)
    patch16.save(path16)

    return 
 
def make_patches_mask(mask,filename, folder):

    kernel = 250
    ## num = 16
    patch1 = mask[0:kernel,0:kernel]
    patch2 = mask[0:kernel,kernel:500]
    patch3 = mask[0:kernel,(kernel+250):750]
    patch4 = mask[0:kernel, (kernel+500):1000]
    patch5 = mask[kernel:500, 0:kernel]
    patch6 = mask[(kernel+250):750, 0:kernel]
    patch7 = mask[(kernel+500):1000, 0:kernel]
    patch8 = mask[kernel:500, kernel:500]
    patch9 = mask[kernel:500, (kernel+250):750]
    patch10 = mask[kernel:500, (kernel+500):1000]
    patch11 = mask[(kernel+250):750, kernel:500]
    patch12 = mask[(kernel+500):1000, kernel:500]
    patch13 = mask[(kernel+250):750, (kernel+250):750]
    patch14 = mask[(kernel+250):750, (kernel+500):1000]
    patch15 = mask[(kernel+500):1000, (kernel+250):750]
    patch16 = mask[(kernel+500):1000,(kernel+500):1000]

    #save the masks in seperate files in a new folder
    path1 = os.path.join(folder,'Annotations',"1_"+filename.split('.')[0]+'.csv')
    np.savetxt(path1, patch1, delimiter=',')
    path2 = os.path.join(folder,'Annotations',"2_"+filename.split('.')[0]+'.csv')
    np.savetxt(path2, patch2, delimiter=',')
    path3 = os.path.join(folder,'Annotations',"3_"+filename.split('.')[0]+'.csv')
    np.savetxt(path3, patch3, delimiter=',')
    path4 = os.path.join(folder,'Annotations',"4_"+filename.split('.')[0]+'.csv')
    np.savetxt(path4, patch4, delimiter=',')
    path5 = os.path.join(folder,'Annotations',"5_"+filename.split('.')[0]+'.csv')
    np.savetxt(path5, patch5, delimiter=',')
    path6 = os.path.join(folder,'Annotations',"6_"+filename.split('.')[0]+'.csv')
    np.savetxt(path6, patch6, delimiter=',')
    path7 = os.path.join(folder,'Annotations',"7_"+filename.split('.')[0]+'.csv')
    np.savetxt(path7, patch7, delimiter=',')
    path8 = os.path.join(folder,'Annotations',"8_"+filename.split('.')[0]+'.csv')
    np.savetxt(path8, patch8, delimiter=',')
    path9 = os.path.join(folder,'Annotations',"9_"+filename.split('.')[0]+'.csv')
    np.savetxt(path9, patch9, delimiter=',')
    path10 = os.path.join(folder,'Annotations',"10_"+filename.split('.')[0]+'.csv')
    np.savetxt(path10, patch10, delimiter=',')
    path11 = os.path.join(folder,'Annotations',"11_"+filename.split('.')[0]+'.csv')
    np.savetxt(path11, patch11, delimiter=',')
    path12 = os.path.join(folder,'Annotations',"12_"+filename.split('.')[0]+'.csv')
    np.savetxt(path12, patch12, delimiter=',')
    path13 = os.path.join(folder,'Annotations',"13_"+filename.split('.')[0]+'.csv')
    np.savetxt(path13, patch13, delimiter=',')
    path14 = os.path.join(folder,'Annotations',"14_"+filename.split('.')[0]+'.csv')
    np.savetxt(path14, patch14, delimiter=',')
    path15 = os.path.join(folder,'Annotations',"15_"+filename.split('.')[0]+'.csv')
    np.savetxt(path15, patch15, delimiter=',')
    path16 = os.path.join(folder,'Annotations',"16_"+filename.split('.')[0]+'.csv')
    np.savetxt(path16, patch16, delimiter=',')
    
    return

In [ ]:
folder = '/content/MonuSeg'
!rm -r MonuSeg

!mkdir 'MonuSeg'
!mkdir 'MonuSeg/Tissue Images'
!mkdir 'MonuSeg/Annotations'

img_directory = '/content/MoNuSegTrainingData/Tissue Images'
mask_directory = '/content/MoNuSegTrainingData/Annotations'

for imgfilename in os.listdir(img_directory):
    img_file = os.path.join(img_directory, imgfilename)
    if(img_file.endswith('.tif')):
      img = Image.open(img_file).convert("RGB")
      nrow, ncol = img.size
      shape = (nrow, ncol)
      make_patches_img(img, imgfilename,folder)
    

for maskfilename in os.listdir(mask_directory):
    mask_file = os.path.join(mask_directory, maskfilename)
    if(mask_file.endswith('.xml')):
      xml_file = mask_file
      mask = generate_masks(shape, xml_file)
      make_patches_mask(mask, maskfilename,folder)

In [ ]:
class Monuseg_Dataset(torch.utils.data.Dataset):
    def __init__(self, root, transforms=None):
        self.root = root
        self.transforms = transforms
        # load all image files, sorting them to
        # ensure that they are aligned
        self.imgs = list(sorted(os.listdir(os.path.join(root, "Tissue Images"))))
        self.masks = list(sorted(os.listdir(os.path.join(root, "Annotations"))))

    def __getitem__(self, idx):
        # load images ad masks
        img_path = os.path.join(self.root, "Tissue Images", self.imgs[idx])
        mask_path = os.path.join(self.root, "Annotations", self.masks[idx])
        #print(mask_path)
        img = Image.open(img_path).convert("RGB")
        img.show()
        #------
        mask = np.loadtxt(mask_path, delimiter=',')

        print(img_path)
        # instances are encoded as different colors
        obj_ids = np.unique(mask)
        # first id is the background, so remove it
        obj_ids = obj_ids[1:]

        # split the color-encoded mask into a set
        # of binary masks
        masks = mask == obj_ids[:, None, None]

        # get bounding box coordinates for each mask
        num_objs = len(obj_ids)
        boxes = []
        for i in range(num_objs):
            pos = np.where(masks[i])
            xmin = np.min(pos[1])
            ymin = np.min(pos[0])
            xmax = np.max(pos[1])
            ymax = np.max(pos[0])
            if(xmin == xmax):
              xmin = xmin-1
              xmax = xmax+1
            if(ymin == ymax):
              ymin = ymin-1
              ymax = ymax+1
            boxes.append([xmin, ymin, xmax, ymax])

        # the coordinates of the N bounding boxes in [x0, y0, x1, y1] format, ranging from 0 to W and 0 to H
        boxes = torch.as_tensor(boxes, dtype=torch.float32) 
        
        # there is only one class
        labels = torch.ones((num_objs,), dtype=torch.int64) # the label for each bounding box
        masks = torch.as_tensor(masks, dtype=torch.uint8) # The segmentation masks for each one of the objects

        image_id = torch.tensor([idx]) #  an image identifier. It should be unique between all the images in the dataset, and is used during evaluation
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0]) # The area of the bounding box
        # suppose all instances are not crowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, boxes,labels,masks,area

    def __len__(self):
        return len(self.imgs)

In [ ]:
import transforms as T
def get_transform(train):
    transforms = []
    # converts the image, a PIL image, into a PyTorch Tensor
    transforms.append(T.ToTensor())
    if train:
        # during training, randomly flip the training images
        # and ground-truth for data augmentation
        transforms.append(T.RandomHorizontalFlip(0.5))

    return T.Compose(transforms)


In [ ]:
dataset = Monuseg_Dataset('/content/MonuSeg')

### Data Processing 


In [ ]:
torch.seed()
from google.colab import drive
drive.mount('/content/drive')


os.chdir('/content/drive/My Drive/MoNuSegTrainingData/')

ROOT_DIR = os.path.abspath("../../")
sys.path.append(ROOT_DIR)

# Data Path
TRAIN_PATH_X = 'MoNuSegTrainingData/Data/'
TRAIN_PATH_Y_Binary = 'MoNuSegTrainingData/Binary_masks/'
TRAIN_PATH_Y_Color = 'MoNuSegTrainingData/Color_masks/'
#TEST_PATH = 'Datasets/test/'
Img_height = 1024
Img_width  = 1024
Img_channel = 3
# Get train and test IDs
train_ids_X = next(os.walk(TRAIN_PATH_X))[2]
train_ids_y_binary = next(os.walk(TRAIN_PATH_Y_Binary))[2]
train_ids_y_color = next(os.walk(TRAIN_PATH_Y_Color))[2]

Img_Train_X = np.zeros((len(train_ids_X), Img_height, Img_width, Img_channel), dtype=np.uint8)
Img_Train_Y_Binary = np.zeros((len(train_ids_y_binary), Img_height, Img_width), dtype=np.bool) ## size?
Img_Train_Y_Color = np.zeros((len(train_ids_y_color), Img_height, Img_width,Img_channel), dtype=np.uint8)

print('Resizing training images and masks')
for n, id in tqdm(enumerate(train_ids_X), total=len(train_ids_X)):
    path = TRAIN_PATH_X
    path1 =  TRAIN_PATH_Y_Binary
    path2 = TRAIN_PATH_Y_Color
    img = imread(path + id)[:, :, :Img_channel]
    img = resize(img, (Img_height, Img_width),
                 mode='constant', preserve_range=True)
    Img_Train_X[n] = img  # Fill empty X_train with values from img
for n, id in tqdm(enumerate(train_ids_y_binary), total=len(train_ids_y_binary)):
    mask = imread(path1 + id)[:, :]
    mask = resize(mask, (Img_height, Img_width),
                 mode='constant', preserve_range=True)
    Img_Train_Y_Binary[n] = mask
for n, id in tqdm(enumerate(train_ids_y_color), total=len(train_ids_y_color)):
    mask = imread(path2 + id)[:, :,:Img_channel]
    mask = resize(mask, (Img_height, Img_width),
                 mode='constant', preserve_range=True)
    Img_Train_Y_Color[n] = mask

#test_ids = next(os.walk(TEST_PATH))[1]
#4/1AY0e-g4YOvl3Q_7zUuXXPXLlUBzchPfETEeUI_0CpsNjPzfBHP5r_M07Jf0

In [ ]:
image_x = random.randint(0, len(train_ids_X))
imshow(Img_Train_X[image_x])
print(Img_Train_X[image_x].shape)
plt.show()
imshow((Img_Train_Y_Binary[image_x]))
print(Img_Train_Y_Binary[image_x].shape)
plt.show()
imshow((Img_Train_Y_Color[image_x]))
plt.show()
print(Img_Train_Y_Color[image_x].shape)

In [ ]:
Img_Train_XTrans = np.zeros(
    (len(train_ids_X), Img_height, Img_width, Img_channel), dtype=np.uint8)
Img_Train_XNorm = np.zeros(
    (len(train_ids_X), Img_height, Img_width, Img_channel), dtype=np.uint8)
transform1 = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
    ),
])

transform = transforms.Compose([
    transforms.ToTensor()
])

for i in range(0,len(Img_Train_X)):
  Img_Train_XTrans = transform(Img_Train_X[i])

len( Img_Train_XTrans )
mean, std = Img_Train_XTrans.mean([1,2]), Img_Train_XTrans.std([1,2])

transform_norm = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])
  
# get normalized image
for i in range(0,len(Img_Train_X)):
  img_normalized= transform_norm(Img_Train_X[i])
  img_normalized = np.array(img_normalized)
  Img_Train_XNorm[i] = img_normalized.transpose(1, 2, 0)

len( Img_Train_XNorm)

image_x = random.randint(0, len(train_ids_X))
imshow(Img_Train_X[image_x])
print(Img_Train_X[image_x].shape)
plt.show()

#imshow(Img_Train_XTrans[image_x])
#print(Img_Train_XTrans[image_x].shape)
#plt.show()

imshow(Img_Train_XNorm[image_x])
print(Img_Train_XNorm[image_x].shape)
plt.show()

Input = []
for i in range(30):
  x = torch.from_numpy(Img_Train_X[i]).float()
  x = x.view(-1,3,1024,1024)
  Input.append(x)
 # t = torch.from_numpy(Img_Train_Y_Binary[i]).float()
  #t = t.view(-1,1,228,228)
  #Target.append(t)

Input = torch.cat(Input,dim = 0)
#Target = torch.cat(Target,dim = 0)
print(Input.shape)
#print(Target.shape)

#Training_set = Img_Train_X[:int(len(Img_Train_X)*0.8)]
#Validation_set = Img_Train_X[int(len(Img_Train_X)*0.8):]


In [ ]:
# Object information: a set of bounding boxes [ymin, xmin, ymax, xmax] and their labels 
bbox0 = []
for i in range(len(dataset)):
  bbox0.append(dataset[i][1])
labels = []
 # 0: background, 1: nuclei
for i in range(len(dataset)):
  labels.append(dataset[i][2])

#Img = cv2.imread(Img_Train_X[image_x])

In [ ]:
bbox = np.concatenate(bbox0, axis=0).astype(int)
bbox = np.asarray(bbox)
print(bbox)
#labels = np.concatenate(labels, axis=1).astype(int)
labels = np.asarray(labels)
print(labels)

### Backbone

In [ ]:
class block(nn.Module):
    def __init__(self, in_channels, intermediate_channels, shortcut = None, stride=1):
        super(block, self).__init__()
        self.expand = 4
        self.conv1 = nn.Conv2d(in_channels, intermediate_channels, kernel_size=1, stride=1, padding=0, bias=True)
        self.bn1 = nn.BatchNorm2d(intermediate_channels)
        self.conv2 = nn.Conv2d(intermediate_channels,intermediate_channels,kernel_size=3,stride=stride,padding=1,bias=True)
        self.bn2 = nn.BatchNorm2d(intermediate_channels)
        self.conv3 = nn.Conv2d(intermediate_channels,intermediate_channels * self.expand,kernel_size=1,stride=1,padding=0,bias=True)
        self.bn3 = nn.BatchNorm2d(intermediate_channels * self.expand)
        self.relu = nn.ReLU()
        self.shortcut = shortcut
        self.stride = 1

    def forward(self, x):
        shortcut = x
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.conv3(x)
        x = self.bn3(x)

        if self.shortcut is not None:
            shortcut = self.shortcut(shortcut)

        x += shortcut
        x = self.relu(x)
        return x


class ResNet_50(nn.Module):
    def __init__(self, block, layers, image_channels):
        super(ResNet_50, self).__init__()
        self.in_channels = 64
        self.layer0 = nn.Sequential(nn.ZeroPad2d(3) , nn.Conv2d(image_channels, 64, kernel_size=7, stride=2,bias=True), nn.BatchNorm2d(64) , 
                                    nn.ReLU(), nn.MaxPool2d(kernel_size=3, stride=2, padding=1))


        self.layer1 = self.make_layer(block, layers[0], intermediate_channels=64, stride=1)
        self.layer2 = self.make_layer(block, layers[1], intermediate_channels=128, stride=2)
        self.layer3 = self.make_layer(block, layers[2], intermediate_channels=256, stride=2)
        self.layer4 = self.make_layer(block, layers[3], intermediate_channels=512, stride=2)


    def forward(self, x):
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        return x

    def stages(self):
        return (self.layer0, self.layer1,self.layer2,self.layer3,self.layer4)

    def make_layer(self, block, num_residual_blocks, intermediate_channels, stride):
        identity_downsample = None
        layers = []

        if stride != 1 or self.in_channels != intermediate_channels * 4:
            identity_downsample = nn.Sequential(nn.Conv2d(self.in_channels,intermediate_channels * 4,kernel_size=1,stride=stride,bias=True),nn.BatchNorm2d(intermediate_channels * 4),)

        layers.append(block(self.in_channels, intermediate_channels, identity_downsample, stride))

        self.in_channels = intermediate_channels * 4

        for i in range(num_residual_blocks - 1):
            layers.append(block(self.in_channels, intermediate_channels))

        return nn.Sequential(*layers)

model = ResNet_50(block, layers = [3,4,6,3],image_channels=3)
output = model.forward(Input)
#output.shape
# from torchsummary import summary
#summary(ResNet_50(block, layers = [3,4,6,3],image_channels=3),(3,250,250))


In [ ]:
# visualize the first 5 channels of the feature maps
imgArray=output.data.cpu().numpy().squeeze(0)
fig=plt.figure(figsize=(12, 4))
figNo = 1
for i in range(5): 
    fig.add_subplot(1, 5, figNo) 
    plt.imshow(imgArray[i], cmap='gray')
    figNo +=1
plt.show()

In [ ]:
class Topdown(nn.Module):
  def __init__(self, C1, C2, C3, C4, C5, out_channels):
    super(Topdown, self).__init__()
    self.out_channels = out_channels
    self.C1 = C1
    self.C2 = C2
    self.C3 = C3
    self.C4 = C4
    self.C5 = C5


    self.p5_conv1 = nn.Conv2d(2048, self.out_channels, kernel_size=1, stride=1,bias=True)  ### pool5
    self.p4_conv1 = nn.Conv2d(1024, self.out_channels, kernel_size=1, stride=1,bias=True)  ## convolving c4
    self.p3_conv1 = nn.Conv2d(512, self.out_channels, kernel_size=1, stride=1,bias=True)   ## convolving c3
    self.p2_conv1 = nn.Conv2d(256, self.out_channels, kernel_size=1, stride=1,bias=True)   ## convolving c2
    self.conv = nn.Conv2d(256, self.out_channels, kernel_size=3, stride=1, padding = 1,bias=True) ## convolution to find features

  def forward(self,x):
       x = self.C1(x)
       x = self.C2(x)
       C2Out = x
       x = self.C3(x)
       C3Out = x
       x = self.C4(x)
       C4Out = x
       x = self.C5(x)
       C5Out = x

       P5 = self.p5_conv1(C5Out)
       M4 = self.p4_conv1(C4Out) + func.upsample(P5, scale_factor = 2)
       P4 = self.conv(M4)
       M3 = self.p3_conv1(C3Out) + func.upsample(P4, scale_factor = 2)
       P3 = self.conv(M3)
       M2 = self.p2_conv1(C2Out) + func.upsample(P3, scale_factor = 2)
       P2 = self.conv(M2)

       return [P2,P3,P4,P5]

#Resnet = ResNet_50(block,layers = [3,4,6,3],image_channels=3)
#C1, C2, C3, C4, C5 = model.stages()

#model1 = Topdown(C1, C2, C3, C4, C5, out_channels = 256)
#P2,P3,P4,P5 = model1.forward(x)
#print(P2.shape)
#print(P3.shape)
#print(P4.shape)
#print(P5.shape)
#from torchsummary import summary
#summary(Topdown(C1, C2, C3, C4, C5, out_channels = 256),(3,250,250))


In [ ]:
# visualize the first 5 channels of the feature maps
imgArray=P5.data.cpu().numpy().squeeze(0)
fig=plt.figure(figsize=(12, 4))
figNo = 1
for i in range(5): 
    fig.add_subplot(1, 5, figNo) 
    plt.imshow(imgArray[i], cmap='gray')
    figNo +=1
plt.show()

###Generate Anchors

In [ ]:
def GenerateAnchor(input_dims,sampling,ratios,scales):

  pixel_size = input_dims//sampling
  ctr_x = np.arange(sampling, (pixel_size+1) * sampling, sampling)
  ctr_y = np.arange(sampling, (pixel_size+1) * sampling, sampling)
  print(len(ctr_x), ctr_x)

  index = 0
  ctr = np.zeros(((pixel_size * pixel_size), 2))
  for x in range(len(ctr_x)):
      for y in range(len(ctr_y)):
          ctr[index, 1] = ctr_x[x] - 8
          ctr[index, 0] = ctr_y[y] - 8
          index +=1
  print(ctr.shape)


  anchor_boxes = np.zeros( ((pixel_size * pixel_size * 9), 4))
  index = 0
  for c in ctr:
      ctr_y, ctr_x = c
      for i in range(len(ratios)):
          for j in range(len(scales)):
              h = sampling * scales[j] * np.sqrt(ratios[i])
              w = sampling * scales[j] * np.sqrt(1./ ratios[i])
              anchor_boxes[index, 0] = ctr_y - h / 2.
              anchor_boxes[index, 1] = ctr_x - w / 2.
              anchor_boxes[index, 2] = ctr_y + h / 2.
              anchor_boxes[index, 3] = ctr_x + w / 2.
              index += 1
  print(anchor_boxes.shape)
  return anchor_boxes

#anchor_boxes = GenerateAnchor(1024,32,[0.5,1,2],[8,16,32])
#print(a.shape)
#print(a)

In [ ]:
def labels_locations(anchor_boxes,bbox,labels):
  ground_box = bbox
  ground_labels = labels

  # Ignore cross-boundary anchor boxes
  # valid anchor boxes with (y1, x1)>0 and (y2, x2)<=1024
  index_inside = np.where((anchor_boxes[:, 0] >= 0) & (anchor_boxes[:, 1] >= 0) & (anchor_boxes[:, 2] <= 1024) &
                (anchor_boxes[:, 3] <= 1024))[0]
  print(index_inside.shape)

  valid_anchor_boxes = anchor_boxes[index_inside]
  print(valid_anchor_boxes.shape)



  # Calculate iou of the valid anchor boxes  
  ious = np.empty((len(valid_anchor_boxes), 2), dtype=np.float32)
  ious.fill(0)
  for num1, i in enumerate(valid_anchor_boxes):
      ya1, xa1, ya2, xa2 = i  
      anchor_area = (ya2 - ya1) * (xa2 - xa1)
      for num2, j in enumerate(ground_box):
          yb1, xb1, yb2, xb2 = j
          box_area = (yb2- yb1) * (xb2 - xb1)
          inter_x1 = max([xb1, xa1])
          inter_y1 = max([yb1, ya1])
          inter_x2 = min([xb2, xa2])
          inter_y2 = min([yb2, ya2])
          if (inter_x1 < inter_x2) and (inter_y1 < inter_y2):
              iter_area = (inter_y2 - inter_y1) * (inter_x2 - inter_x1)
              iou = iter_area / (anchor_area+ box_area - iter_area)            
          else:
              iou = 0.
          ious[num1, num2] = iou
  print(ious.shape)


  # What anchor box has max iou with the ground truth bbox  
  gt_argmax_ious = ious.argmax(axis=0)
  print(gt_argmax_ious)

  gt_max_ious = ious[gt_argmax_ious, np.arange(ious.shape[1])]
  print(gt_max_ious)

  gt_argmax_ious = np.where(ious == gt_max_ious)[0]
  print(gt_argmax_ious)


  # What ground truth bbox is associated with each anchor box 
  argmax_ious = ious.argmax(axis=1)
  print(argmax_ious.shape)
  print(argmax_ious)
  max_ious = ious[np.arange(len(index_inside)), argmax_ious]
  print(max_ious)

  label = np.empty((len(index_inside), ), dtype=np.int32)
  label.fill(-1)
  print(label.shape)



  # Use iou to assign 1 (objects) to two kind of anchors 
  # a) The anchors with the highest iou overlap with a ground-truth-box
  # b) An anchor that has an IoU overlap higher than 0.7 with ground-truth box

  # Assign 0 (background) to an anchor if its IoU ratio is lower than 0.3 for all ground-truth boxes
  pos_iou_threshold  = 0.7
  neg_iou_threshold = 0.3
  label[gt_argmax_ious] = 1
  label[max_ious >= pos_iou_threshold] = 1
  label[max_ious < neg_iou_threshold] = 0


  n_sample = 256
  pos_ratio = 0.5
  n_pos = pos_ratio * n_sample
  print(n_pos)
  pos_index = np.where(label == 1)[0]
  if len(pos_index) > n_pos:
      disable_index = np.random.choice(pos_index, size=(len(pos_index) - n_pos), replace=False)
      label[disable_index] = -1
      
  n_neg = n_sample * np.sum(label == 1)
  print(n_neg)
  neg_index = np.where(label == 0)[0]
  if len(neg_index) > n_neg:
      disable_index = np.random.choice(neg_index, size=(len(neg_index) - n_neg), replace = False)
      label[disable_index] = -1



  # For each valid anchor box, find the groundtruth object which has max_iou 
  max_iou_bbox = ground_box[argmax_ious]
  print(max_iou_bbox.shape)

  # valid anchor boxes h, w, cx, cy 
  height = valid_anchor_boxes[:, 2] - valid_anchor_boxes[:, 0]
  width = valid_anchor_boxes[:, 3] - valid_anchor_boxes[:, 1]
  ctr_y = valid_anchor_boxes[:, 0] + 0.5 * height
  ctr_x = valid_anchor_boxes[:, 1] + 0.5 * width

  # valid anchor box max iou bbox h, w, cx, cy 
  base_height = max_iou_bbox[:, 2] - max_iou_bbox[:, 0]
  base_width = max_iou_bbox[:, 3] - max_iou_bbox[:, 1]
  base_ctr_y = max_iou_bbox[:, 0] + 0.5 * base_height
  base_ctr_x = max_iou_bbox[:, 1] + 0.5 * base_width

  # valid anchor boxes 的 loc = (y-ya/ha), (x-xa/wa), log(h/ha), log(w/wa)
  eps = np.finfo(height.dtype).eps
  height = np.maximum(height, eps)
  width = np.maximum(width, eps)
  dy = (base_ctr_y - ctr_y) / height
  dx = (base_ctr_x - ctr_x) / width
  dh = np.log(base_height / height)
  dw = np.log(base_width / width)
  anchor_locs = np.vstack((dy, dx, dh, dw)).transpose()
  print(anchor_locs.shape)


  anchor_labels = np.empty((len(anchor_boxes),), dtype=label.dtype)
  anchor_labels.fill(-1)
  anchor_labels[index_inside] = label
  anchor_labels = torch.from_numpy(anchor_labels)
  print(anchor_labels.shape)

  anchor_locations = np.empty((len(anchor_boxes),) + anchor_boxes.shape[1:], dtype=anchor_locs.dtype)
  anchor_locations.fill(0)
  anchor_locations[index_inside, :] = anchor_locs
  anchor_locations = torch.from_numpy(anchor_locations)
  print(anchor_locations.shape)

  return anchor_labels,anchor_locations

#anchor_labels,anchor_locs = labels_locations((GenerateAnchor(1024,32,[0.5,1,2],[8,16,32])),bbox0)
#print(a.shape)
#print(b.shape)

### RPN

In [ ]:
class RPN(nn.Module):
  def __init__(self, anchor_no, inchannels,outchannels):
    super(RPN,self).__init__()

    self.anchor_no = anchor_no
    self.conv = nn.Conv2d(inchannels,outchannels,kernel_size=3,stride=1,padding = 1, bias= True)
   #self.conv.weight.data.normal_(0, 0.01)
    self.conv_class = nn.Conv2d(outchannels,self.anchor_no*2,kernel_size = 1, stride = 1,  bias = True)
    #self.conv_class.weight.data.normal_(0, 0.01)
    self.conv_box = nn.Conv2d(outchannels,self.anchor_no*4,kernel_size = 1, stride = 1,  bias = True)
   # self.conv_box.weight.data.normal_(0, 0.01)
    self.softmax = nn.Softmax(dim=2)

  def forward(self,x):

    x = self.conv(x)

    self.region_class = self.conv_class(x)
    
    self.region_class = self.region_class.permute(0,2,3,1)
    self.region_class = self.region_class.contiguous()
    self.region_class = self.region_class.view(x.size()[0], -1, 2)
    self.region_scores_objectiveness =  self.region_class.view(1, 9216, 2)[:, :, 1].contiguous().view(1,-1)

    self.region_class_prob = self.softmax(self.region_class)
   

  

    self.region_box = self.conv_box(x)
    
    self.region_box = self.region_box.permute(0,2,3,1)
    self.region_box = self.region_box.contiguous()
    self.region_box = self.region_box.view(x.size()[0], -1, 4)
    print(self.region_class.shape,self.region_box.shape,self.region_class_prob.shape,self.region_scores_objectiveness.shape)

   
    return [self.region_class,self.region_class_prob,self.region_box,self.region_scores_objectiveness]
#model2 = RPN(9, 256, 512)
#pred_class,pred_prob,pred_box,pred_score = model2.forward(P5)

#print(pred_class)
#print(pred_score)
#print(pred_prob)
#from torchsummary import summary
#summary(RPN(9, 256, 512),(256,50,50))


### RPN Loss

In [ ]:
def rpn_loss(rpn_prob,rpn_box, anchor_locations, anchor_labels):
  rpn_loc = rpn_box[0]
  rpn_score = rpn_prob[0]

  gt_rpn_loc = anchor_locations
  gt_rpn_score = anchor_labels

  print(rpn_loc.shape, rpn_score.shape, gt_rpn_loc.shape, gt_rpn_score.shape)

  # For classification we use cross-entropy loss
  rpn_cls_loss = func.cross_entropy(rpn_score, gt_rpn_score.long(), ignore_index = -1)
  print(rpn_cls_loss)

  # For Regression we use smooth L1 loss as defined
  pos = gt_rpn_score > 0
  mask = pos.unsqueeze(1).expand_as(rpn_loc)
  print(mask.shape)

  # bounding boxes which have positve labels
  mask_loc_preds = rpn_loc[mask].view(-1, 4)
  mask_loc_targets = gt_rpn_loc[mask].view(-1, 4)
  print(mask_loc_preds.shape, mask_loc_targets.shape)

  rpn_loc_loss = func.smooth_l1_loss(mask_loc_preds,mask_loc_targets)
  print(rpn_loc_loss.sum())

  # Combining both the rpn_cls_loss and rpn_reg_loss
  rpn_lambda = 10.
  N_reg = (gt_rpn_score >0).float().sum()
  rpn_loc_loss = rpn_loc_loss.sum() / N_reg
  rpn_loss = rpn_cls_loss + (rpn_lambda * rpn_loc_loss)
  print(rpn_loss)

  return rpn_loss
#Loss = rpn_loss(pred_prob,pred_box,anchor_locs,anchor_labels)
#print(Loss)

### Region Proposal

In [ ]:
def RegionProposal(anchor_boxes, anchor_location, rpn_prob, rpn_box):
  threshold_nms = 0.7  # non-maximum supression (NMS) 
  train_pre_nms = 8000 # no. of train pre-NMS
  train_post_nms = 2000 # after nms, training Fast R-CNN using 2000 RPN proposals
  test_pre_nms = 3000
  test_post_nms = 300 # During testing we evaluate 300 proposals,
  min_size = 32

 
  # format converted from [y1, x1, y2, x2] to [ctr_x, ctr_y, h, w]
  anchor_height = anchor_boxes[:, 2] - anchor_boxes[:, 0]
  anchor_width = anchor_boxes[:, 3] - anchor_boxes[:, 1]
  anchor_ctr_y = anchor_boxes[:, 0] + 0.5 * anchor_height
  anchor_ctr_x = anchor_boxes[:, 1] + 0.5 * anchor_width
  print(anchor_ctr_x.shape)


  # format = (dy, dx, dh, dw)
  rpn_box_numpy = rpn_box.cpu().data.numpy()
  rpn_prob_numpy = rpn_prob.cpu().data.numpy()
  dy = rpn_box_numpy[:, 0::4] # dy
  dx = rpn_box_numpy[:, 1::4] # dx
  dh = rpn_box_numpy[:, 2::4] # dh
  dw = rpn_box_numpy[:, 3::4] # dw
  print(dy.shape)
  dy = torch.from_numpy(dy)
  dy = dy.contiguous().view(-1,1)
  dy = dy.numpy()
  print(dy.shape)
  dx= torch.from_numpy(dx)
  dx = dx.contiguous().view(-1,1)
  dx = dx.numpy()

  dh = torch.from_numpy(dh)
  dh = dh.contiguous().view(-1,1)
  dh = dh.numpy()

  dw = torch.from_numpy(dw)
  dw = dw.contiguous().view(-1,1)
  dw = dw.numpy()

  # ctr_y = dy predicted by RPN * anchor_h + anchor_cy
  # ctr_x similar
  # h = exp(dh predicted by RPN) * anchor_h
  # w similar
  ctr_y = dy * anchor_height[:, np.newaxis] + anchor_ctr_y[:, np.newaxis]
  ctr_x = dx * anchor_width[:, np.newaxis] + anchor_ctr_x[:, np.newaxis]
  h = np.exp(dh) * anchor_height[:, np.newaxis]
  w = np.exp(dw) * anchor_width[:, np.newaxis]
  print(w.shape)
  anchor_location_numpy = anchor_location.numpy()
  # ROI = [y1, x1, y2, x2] 
  roi = np.zeros(rpn_box_numpy.shape, dtype=anchor_location_numpy.dtype)
  print(roi.shape)
  roi = torch.from_numpy(roi)
  roi = roi.contiguous().view(-1,4)
  roi = roi.numpy()

  roi[:, 0::4] = ctr_y - 0.5 * h
  roi[:, 1::4] = ctr_x - 0.5 * w
  roi[:, 2::4] = ctr_y + 0.5 * h
  roi[:, 3::4] = ctr_x + 0.5 * w
  print(roi.shape)

  # clip the predicted boxes to the image
  img_size = (1024, 1024) #Image size
  roi[:, slice(0, 4, 2)] = np.clip(roi[:, slice(0, 4, 2)], 0, img_size[0])
  roi[:, slice(1, 4, 2)] = np.clip(roi[:, slice(1, 4, 2)], 0, img_size[1])
  print(roi.shape, np.max(roi), np.min(roi))


  # Remove predicted boxes with either height or width < threshold.
  hs = roi[:, 2] - roi[:, 0]
  ws = roi[:, 3] - roi[:, 1]
  print(hs.shape,ws.shape)
  keep = np.where((hs >= min_size) & (ws >= min_size))[0] #min_size=16
  print(keep.shape)
  roi = roi[keep, :]
  print(roi.shape)
  score = rpn_prob_numpy[:,keep]
  print(keep.shape, roi.shape, score.shape)
  score = torch.from_numpy(score)
  score = score.view(1,keep.shape[0], 2)[:, :, 1].contiguous().view(1,-1)
  score = score.numpy()
  # Sort all (proposal, score) pairs by score from highest to lowest
  order = score.ravel().argsort()[::-1]
  print(order.shape)

  #Take top pre_nms_topN (e.g. 12000 while training and 300 while testing)
  order = order[:train_pre_nms]
  roi = roi[order, :]
  print(order.shape, roi.shape, roi.shape)

  # Take all the roi boxes [roi_array]
  y1 = roi[:, 0]
  x1 = roi[:, 1]
  y2 = roi[:, 2]
  x2 = roi[:, 3]

  # Find the areas of all the boxes [roi_area]
  areas = (x2 - x1 + 1) * (y2 - y1 + 1)

  #Take the indexes of order the probability score in descending order 
  order = order.argsort()[::-1]
  keep = []
  while (order.size > 0):
      i = order[0] #take the 1st elt in order and append to keep 
      keep.append(i)
      xx1 = np.maximum(x1[i], x1[order[1:]]) 
      yy1 = np.maximum(y1[i], y1[order[1:]])
      xx2 = np.minimum(x2[i], x2[order[1:]])
      yy2 = np.minimum(y2[i], y2[order[1:]])
      w = np.maximum(0.0, xx2 - xx1 + 1)
      h = np.maximum(0.0, yy2 - yy1 + 1)
      inter = w * h
      ovr = inter / (areas[i] + areas[order[1:]] - inter)
      inds = np.where(ovr <= threshold_nms)[0]
      order = order[inds + 1]
  keep = keep[:train_post_nms] # while training/testing , use accordingly
  roi = roi[keep] # the final region proposals
  print(len(keep), roi.shape)

  return roi

#roi = RegionProposal(anchor_boxes, anchor_locs, pred_prob, pred_box)

## What will the input to the pred_prob

### Region Target

In [ ]:
def RegionTargets(bbox,labels,roi,Img):
  n_sample = 128  # Number of samples from roi 
  pos_ratio = 0.25 # Number of positive examples out of the n_samples
  pos_iou_thresh = 0.5  # Min iou of region proposal with any groundtruth object to consider it as positive label
  neg_iou_thresh_high = 0.5  # iou 0~0.5 is considered as negative (0, background)
  neg_iou_thresh_low = 0.0

  # Find the iou of each ground truth object with the region proposals, 
  ious = np.empty((len(roi), 2), dtype=np.float32)
  ious.fill(0)
  for num1, i in enumerate(roi):
      ya1, xa1, ya2, xa2 = i  
      anchor_area = (ya2 - ya1) * (xa2 - xa1)
      for num2, j in enumerate(bbox):
          yb1, xb1, yb2, xb2 = j
          box_area = (yb2- yb1) * (xb2 - xb1)
          inter_x1 = max([xb1, xa1])
          inter_y1 = max([yb1, ya1])
          inter_x2 = min([xb2, xa2])
          inter_y2 = min([yb2, ya2])
          if (inter_x1 < inter_x2) and (inter_y1 < inter_y2):
              iter_area = (inter_y2 - inter_y1) * (inter_x2 - inter_x1)
              iou = iter_area / (anchor_area+ box_area - iter_area)            
          else:
              iou = 0.
          ious[num1, num2] = iou
  print(ious.shape)

  # Find out which ground truth has high IoU for each region proposal, Also find the maximum IoU
  gt_assignment = ious.argmax(axis=1)
  max_iou = ious.max(axis=1)
  print(gt_assignment)
  print(max_iou)

  # Assign the labels to each proposal
  gt_roi_label = labels[gt_assignment]
  print(gt_roi_label)

  # Select the foreground rois as per the pos_iou_thesh and 
  pos_roi_per_image = n_sample*pos_ratio 
  pos_index = np.where(max_iou >= pos_iou_thresh)[0]
  pos_roi_per_this_image = int(min(pos_roi_per_image, pos_index.size))
  if pos_index.size > 0:
      pos_index = np.random.choice(pos_index, size=pos_roi_per_this_image, replace=False)
  print(pos_roi_per_this_image)
  print(pos_index)

  # Similarly for negitive (background) region proposals
  neg_index = np.where((max_iou < neg_iou_thresh_high) &
                              (max_iou >= neg_iou_thresh_low))[0]
  neg_roi_per_this_image = n_sample - pos_roi_per_this_image
  neg_roi_per_this_image = int(min(neg_roi_per_this_image, neg_index.size))
  if  neg_index.size > 0 :
      neg_index = np.random.choice(neg_index, size=neg_roi_per_this_image, replace=False)
  print(neg_roi_per_this_image)
  print(neg_index)

  # display ROI samples with postive 
  img_clone = np.copy(Img)
  for i in range(pos_roi_per_this_image):
      y0, x0, y1, x1 = roi[pos_index[i]].astype(int)
      cv2.rectangle(img_clone, (x0, y0), (x1, y1), color=(255, 255, 2550), thickness=3) 

  for i in range(len(bbox)):
      cv2.rectangle(img_clone, (bbox[i][1], bbox[i][0]), (bbox[i][3], bbox[i][2]), color=(0, 255, 0), thickness=3) # Draw Rectangle
      
  plt.imshow(img_clone)
  plt.show()  

  keep_index = np.append(pos_index, neg_index)
  gt_roi_labels = gt_roi_label[keep_index]
  gt_roi_labels[pos_roi_per_this_image:] = 0  # negative labels --> 0
  sample_roi = roi[keep_index]
  print(sample_roi.shape)

  # Pick the ground truth objects for these sample_roi and 
  # later parameterize as we have done while assigning locations to anchor boxes 
  bbox_for_sampled_roi = bbox[gt_assignment[keep_index]]
  print(bbox_for_sampled_roi.shape)

  height = sample_roi[:, 2] - sample_roi[:, 0]
  width = sample_roi[:, 3] - sample_roi[:, 1]
  ctr_y = sample_roi[:, 0] + 0.5 * height
  ctr_x = sample_roi[:, 1] + 0.5 * width

  base_height = bbox_for_sampled_roi[:, 2] - bbox_for_sampled_roi[:, 0]
  base_width = bbox_for_sampled_roi[:, 3] - bbox_for_sampled_roi[:, 1]
  base_ctr_y = bbox_for_sampled_roi[:, 0] + 0.5 * base_height
  base_ctr_x = bbox_for_sampled_roi[:, 1] + 0.5 * base_width

  eps = np.finfo(height.dtype).eps
  height = np.maximum(height, eps)
  width = np.maximum(width, eps)

  dy = (base_ctr_y - ctr_y) / height
  dx = (base_ctr_x - ctr_x) / width
  dh = np.log(base_height / height)
  dw = np.log(base_width / width)

  gt_roi_locs = np.vstack((dy, dx, dh, dw)).transpose()
  print(gt_roi_locs.shape)

  return sample_roi,gt_roi_locs

#roi_labels,roi_locs = RegionTargets(bbox0,labels,roi,Img_Train_X[image_x])
#roi_labels = torch.from_numpy(roi_labels)
#roi_locs = torch.from_numpy(roi_locs)

### Roi Align

In [ ]:
def ROI_ALIGN(out_dims,inputs,roi_labels):
  roialign = []
  for i in range(len(inputs)):
    feature_maps = inputs[i]
    roi_align_obj = RoIAlign(out_dims, 2**-5, sampling_ratio= 4, aligned=False)
    roi_align = roi_align_obj(feature_maps.type(torch.float32), [roi_labels.type(torch.float32)])
    #roi_align = roi_align.detach().numpy()
    roialign.append(roi_align)
    #roi_align = torch.from_numpy(roi_align)
    # Pack pooled features into one tensor
  roialign = torch.cat(roialign, dim=0)

  print(roialign.shape)
  return roialign
#eature_maps = [P2,P3,P4,P5]
#roi_align1 = ROI_ALIGN(7,feature_maps,roi_labels)

### ROI Allign

In [ ]:
def bilinear_interpolate(img, x, y):
    x0 = torch.floor(x).type(torch.int64)
    x1 = x0 + 1

    y0 = torch.floor(y).type(torch.int64)
    y1 = y0 + 1

    x0 = torch.clamp(x0, 0, img.shape[1]-1)
    x1 = torch.clamp(x1, 0, img.shape[1]-1)
    y0 = torch.clamp(y0, 0, img.shape[0]-1)
    y1 = torch.clamp(y1, 0, img.shape[0]-1)

    Ia = img[y0, x0]
    Ib = img[y1, x0]
    Ic = img[y0, x1]
    Id = img[y1, x1]

    norm_const = 1/((x1.type(torch.float32) - x0.type(torch.float32))*(y1.type(torch.float32) - y0.type(torch.float32)))

    wa = (x1.type(torch.float32) - x) * (y1.type(torch.float32) - y) * norm_const
    wb = (x1.type(torch.float32) - x) * (y-y0.type(torch.float32)) * norm_const
    wc = (x-x0.type(torch.float32)) * (y1.type(torch.float32) - y) * norm_const
    wd = (x-x0.type(torch.float32)) * (y - y0.type(torch.float32)) * norm_const

    return torch.t(torch.t(Ia)*wa) + torch.t(torch.t(Ib)*wb) + torch.t(torch.t(Ic)*wc) + torch.t(torch.t(Id)*wd)


In [ ]:
class TorchROIAlign(nn.Module):

    def __init__(self, output_size, scaling_factor):
        self.output_size = output_size
        self.scaling_factor = scaling_factor

    def _roi_align(self, features, scaled_proposal):
        _, num_channels, h, w = features.shape

        xp0, yp0, xp1, yp1 = scaled_proposal
        p_width = xp1 - xp0
        p_height = yp1 - yp0

        w_stride = p_width/self.output_size
        h_stride = p_height/self.output_size

        interp_features = torch.zeros((num_channels, self.output_size, self.output_size))

        for i in range(self.output_size):
            for j in range(self.output_size):
                x_strt = i*w_stride + xp0
                y_strt = j*h_stride + yp0

                # generate 4 points for interpolation
                # notice no rounding
                x1 = torch.Tensor([x_strt + 0.25*w_stride])
                x2 = torch.Tensor([x_strt + 0.75*w_stride])
                y1 = torch.Tensor([y_strt + 0.25*h_stride])
                y2 = torch.Tensor([y_strt + 0.75*h_stride])

                for c in range(num_channels):
                    img = features[0,c]
                    v1 = bilinear_interpolate(img, x1, y1)
                    v2 = bilinear_interpolate(img, x1, y2)
                    v3 = bilinear_interpolate(img, x2, y1)
                    v4 = bilinear_interpolate(img, x2, y2)

                    interp_features[c, j, i] = (v1+v2+v3+v4)/4
        
        return interp_features

    def __call__(self, feature_layer, proposals):

        _, num_channels, _, _ = feature_layer.shape
        # first scale proposals down by self.scaling factor
        scaled_proposals = torch.zeros_like(proposals)

        # notice no ceil or floor functions
        scaled_proposals[:, 0] = proposals[:, 0] * self.scaling_factor
        scaled_proposals[:, 1] = proposals[:, 1] * self.scaling_factor
        scaled_proposals[:, 2] = proposals[:, 2] * self.scaling_factor
        scaled_proposals[:, 3] = proposals[:, 3] * self.scaling_factor

        res = torch.zeros((len(proposals), num_channels, self.output_size,
                        self.output_size))
        for idx in tqdm(range(len(scaled_proposals))):
            proposal = scaled_proposals[idx]
            res[idx] = self._roi_align(feature_layer, proposal)

        return res

my_roi_align_obj = TorchROIAlign(7, 2**(-5))
feat_layer = torch.randn(1, 64, 32, 32)
roi_align1 = my_roi_align_obj(P5,roi_labels)
print(roi_align1)
print(roi_align1.shape)

###MCRNN

In [ ]:
class classify(nn.Module):
  def __init__(self, inchannels, num_classes):
    super(classify,self).__init__()


    self.conv1 = nn.Conv2d(inchannels,1024,kernel_size= 3,stride=1,bias= True)
    self.bn = nn.BatchNorm2d(1024)
    self.relu = nn.ReLU()
    self.conv2 = nn.Conv2d(1024,1024,kernel_size= 1,stride = 1, bias = True)

    self.classify_box = nn.Linear(1024,num_classes*4)
    self.softmax = nn.Softmax(dim=1)
    self.classify_classes = nn.Linear(1024, num_classes)

  def forward(self,x):
     x = self.conv1(x)
     x = self.bn(x)
     x = self.relu(x)
     x = self.conv2(x)
     x = self.bn(x)
     x = self.relu(x)

     x = x.view(-1,1024)
     mrcnn_class = self.classify_classes(x)
     mrcnn_class_prob = self.softmax(mrcnn_class)

     mrcnn_box = self.classify_box(x)
     mrcnn_box = mrcnn_box.view(mrcnn_box.size()[0], -1, 4)

     return mrcnn_class, mrcnn_class_prob, mrcnn_box

#roi_align_obj_1 = RoIAlign(7, 2**-5, sampling_ratio= 4, aligned=False)
#roi_align1 = roi_align_obj_1(P5.type(torch.float32), [roi_labels.type(torch.float32)])
#print(roi_align1.shape)
#model7 = classify(256,2)

#mrcnn_class, mrcnn_class_prob, mrcnn_box =  model7.forward(roi_align1)

#print(mrcnn_class.shape)
#print( mrcnn_class_prob.shape)
#print( mrcnn_box)
#summary(classify(256,10),(256,7,7))

In [ ]:
# display bounding box and labels
img0_clone = np.copy(Img_Train_X[image_x])
mrcnn_box = mrcnn_box.view(mrcnn_box.size(0),-1)
for i in range(len(mrcnn_box)):
    cv2.rectangle(img0_clone, (mrcnn_box[i][1], mrcnn_box[i][0]), (mrcnn_box[i][3], mrcnn_box[i][2]), color=(255, 255, 2550), thickness=5) 
    #cv2.putText(img0_clone, str(int(labels[i])), (bbox0[i][3],bbox0[i][2]), cv2.FONT_HERSHEY_SIMPLEX, 3, (0,0,255),thickness=3) 
plt.imshow(img0_clone)
plt.show()  

In [ ]:
class Mask_segmentation(nn.Module):
    def __init__(self, inchannels, num_classes):
        super(Mask_segmentation, self).__init__()
        self.conv1 = nn.Conv2d(inchannels, 256, kernel_size=3, stride=1,padding = 1, bias = True)
        self.bn = nn.BatchNorm2d(256)
        self.conv2 = nn.Conv2d(256, 256, kernel_size=3, padding = 1, stride=1,bias = True)
        self.deconv = nn.ConvTranspose2d(256, 256, kernel_size=2, stride=2,bias = True)
        self.conv3 = nn.Conv2d(256, num_classes, kernel_size=1, stride=1, bias = True)
        self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn(x)
        x = self.relu(x)
        x = self.deconv(x)
        x = self.relu(x)
        x = self.conv3(x)
        x = self.sigmoid(x)

        return x
#roi_align_obj = RoIAlign(14, 2**-5, sampling_ratio= 4, aligned=False)
#roi_align2 = roi_align_obj(P5.type(torch.float32), [roi_labels.type(torch.float32)])
#feature_maps = [P2,P3,P4,P5]
#roi_align2 = ROI_ALIGN(14,feature_maps,roi_labels)
#print(roi_align2.shape)
#mcrnn_seg = Mask_segmentation(256,80)
#mcrnn_mask  = mcrnn_seg.forward(roi_align2)
#print(mcrnn_mask.shape)
#from torchsummary import summary
#summary(Mask_segmentation(256,80),(256,14,14))

In [ ]:
targets = []
for i in range(len(dataset)):
  targets.append(dataset[i][3])


In [ ]:
def compute_mrcnn_class_loss(roi_labels,mrcnn_class):

    # Loss
    loss = func.cross_entropy(mrcnn_class,roi_labels.long())
    return loss


def compute_mrcnn_box_loss(roi_locs, roi_labels,mrcnn_box):

    if roi_labels.size():
        # Only positive ROIs contribute to the loss. And only
        # the right class_id of each ROI. Get their indicies.
        positive_roi_ix = torch.nonzero(roi_labels > 0)[:, 0]
        positive_roi_class_ids = roi_labels[positive_roi_ix.data].long()
        indices = torch.stack((positive_roi_ix,positive_roi_class_ids), dim=1)
        roi_locs = roi_locs[indices[:,0].data,:]
        mrcnn_box = mrcnn_box[indices[:,0].data,indices[:,1].data,:]

        # Smooth L1 loss
        loss = func.smooth_l1_loss(mrcnn_box, roi_locs)
    
    return loss

def compute_mrcnn_mask_loss(target_masks, roi_labels, mrcnn_mask):
  
    positive_ix = torch.nonzero(roi_labels > 0)[:, 0]
    positive_class_ids = roi_labels[positive_ix.data].long()
    indices = torch.stack((positive_ix, positive_class_ids), dim=1)

    y_true = target_masks[indices[:,0].data,:,:]
    y_pred = mrcnn_mask[indices[:,0].data,indices[:,1].data,:,:]

    loss = func.binary_cross_entropy(y_pred, y_true)

    return loss


#loss = compute_mrcnn_class_loss(roi_labels,mrcnn_class) + compute_mrcnn_box_loss(roi_locs, roi_labels,mrcnn_box) + compute_mrcnn_mask_loss(target, roi_labels, mrcnn_mask)

In [ ]:
def MRCNN(image,bbox,labels):

  def initialize_weights(self):
    for m in self.modules():
      if isinstance(m, nn.Conv2d):
        nn.init.xavier_uniform(m.weight)
      if m.bias is not None:
        m.bias.data.zero_()
      elif isinstance(m, nn.BatchNorm2d):
        m.weight.data.fill_(1)
        m.bias.data.zero_()
      elif isinstance(m, nn.Linear):
        m.weight.data.normal_(0, 0.01)
        m.bias.data.zero_()
  # Object information: a set of bounding boxes [ymin, xmin, ymax, xmax] and their labels 
  Resnet = ResNet_50(block, layers = [3,4,23,3],image_channels=3)
  #output = model.forward(x)
  C1, C2, C3, C4, C5 = Resnet.stages()

  fpn = Topdown(C1, C2, C3, C4, C5, out_channels = 256)
  
  #P2,P3,P4,P5 = fpn.forward(x)

  anchor_boxes = GenerateAnchor(1024,32,[0.5,1,2],[8,16,32])

  anchor_labels,anchor_locs = labels_locations(anchor_boxes,bbox,labels)

  rpn = RPN(9, 256, 512)

  [P2,P3,P4,P5] = fpn(image)

  feature_maps = [P2,P3,P4,P5]

        # Loop through pyramid layers
  layer_outputs = []  # list of lists
  for p in feature_maps:
    layer_outputs.append(rpn(p))
  outputs = list(zip(*layer_outputs))
  outputs = [torch.cat(list(o), dim=1) for o in outputs]
  pred_class,pred_prob,pred_box,pred_score = outputs

  roi = RegionProposal(anchor_boxes, anchor_locs, pred_prob, pred_box)

  roi_labels,roi_locs = RegionTargets(bbox,labels,roi,image)
  roi_labels = torch.from_numpy(roi_labels)
  roi_locs = torch.from_numpy(roi_locs) 

  roi_align1 = ROI_ALIGN(7,feature_maps,roi_labels)
  mrcnn_classify = classify(256,2)
  mrcnn_class, mrcnn_class_prob, mrcnn_box =   mrcnn_classify .forward(roi_align1)

  roi_align2 = ROI_ALIGN(14,feature_maps,roi_labels)
  mcrnn_seg = Mask_segmentation(256,2)
  mcrnn_mask  = mcrnn_seg.forward(roi_align2)
  

  rpn_Loss = rpn_loss(pred_prob,pred_box,anchor_locs,anchor_labels)
  mrcnn_loss = compute_mrcnn_class_loss(roi_labels,mrcnn_class) + compute_mrcnn_box_loss(roi_locs, roi_labels,mrcnn_box) + compute_mrcnn_mask_loss(targets, roi_labels, mrcnn_mask)

  
  return rpn_Loss ,mrcnn_loss


In [ ]:
MRCNN(Img_Train_X[0],bbox = np.array([[160, 147, 260, 234], [139, 312, 200, 348]]),labels = np.array([1, 1]))

In [ ]:
params = []
params.append(Resnet.parameters())
params.append(fpn.parameters())
params.append(rpn.parameters())
params.append(mcrnn_seg.parameters())
  ## How to get params
optimizer = torch.optim.Adam(params, lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)



In [ ]:
import utils
torch.manual_seed(1)
dataset = Monuseg_Dataset('/content/MonuSeg',get_transform(train = True))
dataset_test = Monuseg_Dataset('/content/MonuSeg', get_transform(train=True))

indices = torch.randperm(len(dataset)).tolist()

dataset = torch.utils.data.Subset(dataset, indices[0:450])

dataset_test = torch.utils.data.Subset(dataset_test, indices[450:480])


In [ ]:
#Backpropagation
for i in range(10):
  loss = MRCNN(dataset[0:10])
  loss.backward()
  optimizer.step()
  optimizer.zero_grad()

In [ ]:
## Size of target image
## How to train the model(Like Implementation?)
## How to decide the bouding boxes(min and max coordinates)
## How to obatin the image from masks - (PIL Library)
## The model is not running the session crashes
## Which classes?
## Loss fucntions
